In [1]:
# Import libs
import sys
import matplotlib.pyplot as plt
from docplex.mp.model import Model
from docplex.util.environment import get_environment
from docplex.mp.relax_linear import LinearRelaxer
import matplotlib.patches as mpatches

try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')
     

In [ ]:
# TELAS
A = 'Voile'
B = 'Rustico'
telas = [A,B]
LIMITE_PRODUCCION = 150

# PRECIOS VENTA
PRECIO_VENTA = {
    A: 420,
    B: 873,
}

# COSTOS
MT = 30
COSTO_FABRICACION = {
    A: 9*MT,
    B: 10*MT
}

COSTO_MATERIA_PRIMA = {
    A: 3*MT,
    B: 4*MT
}

COSTO_VENTA = {
    A: 20,
    B: 25
}

COSTO_COMPRA = {
    A: 430,
    B: 830
}

# STOCK
STOCK_INICIAL = {
    A: 30,
    B: 150
}

LIMITE_STOCK = {
    A: 50, #Como el máximo es 2*min, no son necesarios ambos valores
    B: 120
}
COSTO_EMBALAJE = 380
LIMITE_EMBALAJE = 950

# DEMANDA
LIMITES_DEMANDA = {
    A: (100, 800),
    B: (150, 300)
}

# VARIABLES FINANCIERAS
DISPONIBILIDAD = 50_000
LIMITES_EXCEDENTE = [0, 50_000, 150_000]
INTERVALOS_EXCEDENTE = len(LIMITES_EXCEDENTE)
PORCENTAJE_EXCEDENTE = {
    1: 0.052,
    2: 0.054
}
LIMITES_DEFICIT = [0, 50_000, 100_000]
INTERVALOS_DEFICIT = len(LIMITES_DEFICIT)
PORCENTAJE_DEFICIT = {
    1: 0.056,
    2: 0.06
}
AHORRO = 35_000
LIM_AHORRO = 350
     

In [ ]:
mdl = Model(name="Ejericicio 3.15: Producción de telas") 

# Variables de produccion
tela_producida = {}
bobina_comprada = {}
bobina_vendida = {}
stock_final = {}
for tela in telas:
    tela_producida[tela] = mdl.continuous_var(name=f'tela_producida_{tela}')
    bobina_comprada[tela] = mdl.continuous_var(name=f'bobina_{tela}_comprada')
    bobina_vendida[tela] = mdl.continuous_var(name=f'bobina_{tela}_vendida')
    stock_final[tela] = mdl.continuous_var(name=f'stock_final_de_{tela}')
voile_embaladas = mdl.continuous_var(name=f'bobinas_Voile_embaladas_extra')
# Variables financieras
excedente_i = {intervalo: mdl.continuous_var(name=f'excedente_de_intervalo_{intervalo}') for intervalo in range(1, INTERVALOS_EXCEDENTE)}
excedente = mdl.continuous_var(name=f'excedente')
deficit_i = {intervalo: mdl.continuous_var(name=f'deficit_de_intervalo_{intervalo}') for intervalo in range(1, INTERVALOS_DEFICIT)}
deficit = mdl.continuous_var(name=f'deficit')
# Variables binarias
se_produce = {tela: mdl.binary_var(name=f'se_produce_tela_{tela}') for tela in telas}
es_excedente_i = {intervalo: mdl.binary_var(name=f'es_excedente_{intervalo}') for intervalo in range(1, INTERVALOS_EXCEDENTE)}
es_deficit_i = {intervalo: mdl.binary_var(name=f'es_deficit_{intervalo}') for intervalo in range(1, INTERVALOS_DEFICIT)}
embalaje_extra = mdl.binary_var(name=f'se_compra_de_segundo_proveedor')
se_ahorra = mdl.binary_var(name=f'se_ahorra_la_suma_{AHORRO}')

ingresos = mdl.sum(PRECIO_VENTA[tela]*bobina_vendida[tela] for tela in telas)
costos_produccion = mdl.sum((COSTO_FABRICACION[tela] + COSTO_MATERIA_PRIMA[tela])*tela_producida[tela] for tela in telas)
costos_compra = mdl.sum(COSTO_COMPRA[tela]*bobina_comprada[tela] for tela in telas)
costos_venta = COSTO_VENTA['Voile']*bobina_vendida['Voile'] + COSTO_VENTA['Rustico']*bobina_vendida['Rustico'] 
+ COSTO_EMBALAJE*voile_embaladas
ganancia_excedente = mdl.sum(excedente_i[intervalo]*PORCENTAJE_EXCEDENTE[intervalo] for intervalo in range(1, INTERVALOS_EXCEDENTE))
interes_deficit = mdl.sum(deficit_i[intervalo]*PORCENTAJE_DEFICIT[intervalo] for intervalo in range(1, INTERVALOS_DEFICIT))
ahorro = AHORRO*se_ahorra

# --Restricciones--
# Balance vendidas + stock final = producidas + compradas + stock inicial
mdl.add_constraint(
    bobina_vendida['Rustico'] + stock_final['Rustico'] == tela_producida['Rustico'] + bobina_comprada['Rustico'] + STOCK_INICIAL['Rustico']
    )
mdl.add_constraint(
    bobina_vendida['Voile'] + stock_final['Voile'] == tela_producida['Voile'] + voile_embaladas + bobina_comprada['Voile'] + STOCK_INICIAL['Voile']
    )
# Demanda
for tela in telas:
    mdl.add_constraint(bobina_vendida[tela] >= LIMITES_DEMANDA[tela][0])
    mdl.add_constraint(bobina_vendida[tela] <= LIMITES_DEMANDA[tela][1])
# Stock
for tela in telas:
    mdl.add_constraint(stock_final[tela] >= LIMITE_STOCK[tela])
    mdl.add_constraint(stock_final[tela] <= 2 * LIMITE_STOCK[tela])
# Financiero
mdl.add_constraint(
    DISPONIBILIDAD + ingresos - 0.5*(costos_compra + costos_produccion + costos_venta) == DISPONIBILIDAD + excedente - deficit
    )
mdl.add_constraint(excedente == mdl.sum(excedente_i[intervalo] for intervalo in range(1, INTERVALOS_EXCEDENTE)))
mdl.add_constraint(deficit == mdl.sum(deficit_i[intervalo] for intervalo in range(1, INTERVALOS_DEFICIT)))
for i in range(1,INTERVALOS_EXCEDENTE):
    mdl.add_constraint(excedente_i[i] >= (LIMITES_EXCEDENTE[i-1]+0.1) * es_excedente_i[i])
    mdl.add_constraint(excedente_i[i] <= LIMITES_EXCEDENTE[i] * es_excedente_i[i])
mdl.add_constraint(mdl.sum(es_excedente_i[intervalo] for intervalo in range(1, INTERVALOS_EXCEDENTE)) <= 1)
for i in range(1,INTERVALOS_DEFICIT):
    mdl.add_constraint(deficit_i[i] >= (LIMITES_DEFICIT[i-1]+0.1) * es_deficit_i[i])
    mdl.add_constraint(deficit_i[i] <= LIMITES_DEFICIT[i] * es_deficit_i[i])
mdl.add_constraint(mdl.sum(es_deficit_i[intervalo] for intervalo in range(1, INTERVALOS_DEFICIT)) <= 1)
# Si se compra, no se produce
M1 = 1000
for tela in telas:
    mdl.add_constraint(bobina_vendida[tela] == bobina_comprada[tela] + tela_producida[tela])
    mdl.add_constraint(tela_producida[tela] >= LIMITE_PRODUCCION * se_produce[tela])
    mdl.add_constraint(tela_producida[tela] <= M1 * se_produce[tela])
    mdl.add_constraint(bobina_comprada[tela] <= M1*(1 - se_produce[tela]))
# Ahorro
mdl.add_constraint(mdl.sum(tela_producida[tela] for tela in telas) >= LIM_AHORRO * se_ahorra)
# Voile embaladas
M2 = 1000
mdl.add_constraint(
    mdl.sum(bobina_vendida[tela] for tela in telas) + voile_embaladas >= LIMITE_EMBALAJE
)
mdl.add_constraint(
    voile_embaladas <= M2 * embalaje_extra
)
mdl.add_constraint(
    voile_embaladas >= (LIMITE_EMBALAJE - mdl.sum(bobina_vendida[tela] for tela in telas)) * embalaje_extra
)
# Funcion objetivo
beneficio = ingresos - costos_produccion - costos_compra - costos_venta + ganancia_excedente - interes_deficit - ahorro
solution = mdl.maximize(beneficio)
     